<a href="https://colab.research.google.com/github/Farah14-lab/MachineLearning-BigData/blob/main/ML_BigData_Slide30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
# Connect Google Drive Untuk Ambil Data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
# Install PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
# Import Library
from pyspark.sql import SparkSession, Row
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType, StructField, StructType
from pyspark.ml.recommendation import ALSModel

spark = SparkSession.builder.appName("Movie Lens").getOrCreate()

In [78]:
# Define the parseRating function
def parseRating(line):
    if line != "userId,movieId,rating,timestamp":
        fields = line.split(",")
        userId = int(fields[0])
        movieId = int(fields[1])
        rating = float(fields[2])
        timestamp = int(fields[3])
        return Row(userId=userId, movieId=movieId, rating=rating, timestamp=timestamp)

In [79]:
# Read the file and parse the ratings
raw = spark.sparkContext.textFile("/content/drive/MyDrive/BigData/ML/ratings.DAT")
ratings = raw.map(parseRating).filter(lambda x: x is not None)

In [80]:
# Define the schema for DataFrame
schema = StructType([
    StructField("userId", IntegerType(), nullable=True),
    StructField("movieId", IntegerType(), nullable=True),
    StructField("rating", FloatType(), nullable=True),
    StructField("timestamp", IntegerType(), nullable=True)
])

In [81]:
# Create DataFrame using the defined schema
ratings_df = spark.createDataFrame(ratings, schema)

In [82]:
# Display the DataFrame
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [83]:
# Data Training 80% dan Test 20%
training, test = ratings.randomSplit([0.8, 0.2])

In [84]:
# Membuat Model
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(ratings_df)  # Melatih model dengan DataFrame yang sesuai

# Menyimpan model dengan mengatasi konflik
model.write().overwrite().save("mymodel")

# Memuat model dari file yang telah disimpan
model = ALSModel.load("mymodel")

# Prediksi Data
predictions = model.transform(test_df)
mse = predictions.withColumn("diff", col("rating") - col("prediction")).select((col("diff") ** 2).alias("squared_diff")).filter(~col("squared_diff").isNull()).agg({"squared_diff": "sum"}).collect()[0][0]
print("Mean Squared Error:", mse)

predictions.show(10)



NameError: ignored